In [1]:
# Load the necessary libraries and data again
import pandas as pd
from datetime import timedelta

# Load the CSV data
data = pd.read_csv('./ranked_by_data.csv')

# Initialize empty lists to hold the date, cryptocurrency name, and market cap
dates = []
cryptos = []
market_caps = []

# Loop over each date column in the dataframe
for col in data.columns[1:]:
    # Loop over each row in the column
    for val in data[col]:
        # Split the tuple into its two components
        crypto, market_cap = eval(val)
        # Append the data to the lists
        dates.append(col)
        cryptos.append(crypto)
        market_caps.append(float(market_cap))

# Create a new dataframe with the transformed data
df = pd.DataFrame({
    'Date': pd.to_datetime(dates),
    'Crypto': cryptos,
    'MarketCap': market_caps
})

df = df.sort_values(by=['Date', 'MarketCap'], ascending=[True, False])

# Calculate the rank of each cryptocurrency on each date
df['Rank'] = df.groupby('Date')['MarketCap'].rank(method='min', ascending=False)

NameError: name 'get_moving_cryptos' is not defined

In [2]:
df.head()

,Date,Crypto,MarketCap,Rank
0,2017-01-02,bitcoin,1.638793e+10,1.0
1,2017-01-02,ethereum,7.318175e+08,2.0
2,2017-01-02,ripple,2.292520e+08,3.0
3,2017-01-02,litecoin,2.251494e+08,4.0
4,2017-01-02,monero,2.188227e+08,5.0


In [30]:
# Function to get the cryptocurrencies that entered the top x*100 from outside the top y*100 in a given week
def get_gaining_rank_by_cryptos_old(x, start_date):

    # Convert start_date to datetime
    start_date = pd.to_datetime(start_date)
    
    # Calculate end date
    end_date = start_date + timedelta(days=6)
    
    # Filter the data to only include the specified week
    week_data = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

    # Find the cryptocurrencies that were outside the top y*100 at the start of the week
    outside_top_y_start = week_data[week_data['Date'] == start_date]
    outside_top_y_start = outside_top_y_start[outside_top_y_start['Rank'] > x*100]['Crypto'].unique()

    # Find the cryptocurrencies that were inside the top x*100 at the end of the week
    inside_top_x_end = week_data[week_data['Date'] == end_date]
    inside_top_x_end = inside_top_x_end[inside_top_x_end['Rank'] <= x*100]['Crypto'].unique()

    # Find the cryptocurrencies that were outside the top y*100 at the start of the week but inside the top x*100 at the end
    entered_top_x = [crypto for crypto in outside_top_y_start if crypto in inside_top_x_end]

    # Get previous rank and end rank for each crypto that entered the top x*100
    crypto_ranks = {crypto: (week_data[(week_data['Crypto'] == crypto) & (week_data['Date'] == start_date)]['Rank'].values[0], 
                              week_data[(week_data['Crypto'] == crypto) & (week_data['Date'] == end_date)]['Rank'].values[0]) for crypto in entered_top_x}

    return {k: v for k, v in sorted(crypto_ranks.items(), key=lambda item: item[1][1])}  # Sort by end rank

In [31]:
# Redefine the function to include the times of ranks
def get_gaining_rank_by_cryptos(x, start_date):
    # Convert start_date to datetime
    start_date = pd.to_datetime(start_date)
    
    # Calculate end date
    end_date = start_date + timedelta(days=6)
    
    # Filter the data to only include the specified week
    week_data = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

    # Find the cryptocurrencies that were outside the top y*100 at the start of the week
    outside_top_y_start = week_data[week_data['Date'] == start_date]
    outside_top_y_start = outside_top_y_start[outside_top_y_start['Rank'] > x*100]['Crypto'].unique()

    # Find the cryptocurrencies that were inside the top x*100 at the end of the week
    inside_top_x_end = week_data[week_data['Date'] == end_date]
    inside_top_x_end = inside_top_x_end[inside_top_x_end['Rank'] <= x*100]['Crypto'].unique()

    # Find the cryptocurrencies that were outside the top y*100 at the start of the week but inside the top x*100 at the end
    entered_top_x = [crypto for crypto in outside_top_y_start if crypto in inside_top_x_end]

    # Get previous rank and end rank for each crypto that entered the top x*100
    crypto_ranks = {crypto: {str(start_date.date()): week_data[(week_data['Crypto'] == crypto) & (week_data['Date'] == start_date)]['Rank'].values[0], 
                              str(end_date.date()): week_data[(week_data['Crypto'] == crypto) & (week_data['Date'] == end_date)]['Rank'].values[0]} for crypto in entered_top_x}

    return {k: v for k, v in sorted(crypto_ranks.items(), key=lambda item: item[1][str(end_date.date())])}  # Sort by end rank


In [24]:
# Run the function with x = 2 and the third week of January, 2023
get_gaining_rank_by_cryptos(2, '2023-02-15')

{'conflux-token': (253.0, 78.0),
 'access-protocol': (889.0, 141.0),
 'biconomy': (205.0, 192.0),
 'alchemy-pay': (340.0, 197.0),
 'nervos-network': (245.0, 198.0)}

In [13]:
# Gaining rank: outside of top x*100 to inside top x*100
     # Before: Rank > x * 100
     # After:  Rank <= x * 100
# Gaining rank: outside of top x*100 to between x*100 and (x-1)*100
     # Before: Rank > x * 100
     # After: (x-1)*100 < Rank <= x * 100
# Losing rank: top x*100 to outside of top x*100
     # Before: Rank <= x * 100
     # After:  Rank > x * 100
# Losing rank: top x*100 to between top x*100 and (x+1)*100
     # Before: Rank <= x * 100
     # After: x*100 > Rank >= (x+1) * 100
# Top Coins by gained rank in a given period
     # Sort the coins by the increase in rank in a given period (from_week, to_week)
# Top Coins by lost rank in a given period
    # Sort the coins by the decrese in rank in a given period (from_week, to_week)

In [15]:
# Function 2: Gaining rank: outside of top x*100 to between x*100 and (x-1)*100
def get_gaining_rank_by_cryptos_2(x, start_date):
    # Convert start_date to datetime
    start_date = pd.to_datetime(start_date)
    
    # Calculate end date
    end_date = start_date + timedelta(days=6)
    
    # Filter the data to only include the specified week
    week_data = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

    # Find the cryptocurrencies that were outside the top y*100 at the start of the week
    outside_top_y_start = week_data[week_data['Date'] == start_date]
    outside_top_y_start = outside_top_y_start[outside_top_y_start['Rank'] > x*100]['Crypto'].unique()

    # Find the cryptocurrencies that were inside the top x*100 and outside (x-1)*100 at the end of the week
    inside_top_x_end = week_data[week_data['Date'] == end_date]
    inside_top_x_end = inside_top_x_end[(inside_top_x_end['Rank'] <= x*100) & (inside_top_x_end['Rank'] > (x-1)*100)]['Crypto'].unique()

    # Find the cryptocurrencies that were outside the top y*100 at the start of the week but inside the top x*100 at the end
    entered_top_x = [crypto for crypto in outside_top_y_start if crypto in inside_top_x_end]

    # Get previous rank and end rank for each crypto that entered the top x*100
    crypto_ranks = {crypto: (week_data[(week_data['Crypto'] == crypto) & (week_data['Date'] == start_date)]['Rank'].values[0], 
                              week_data[(week_data['Crypto'] == crypto) & (week_data['Date'] == end_date)]['Rank'].values[0]) for crypto in entered_top_x}

    return {k: v for k, v in sorted(crypto_ranks.items(), key=lambda item: item[1][1])}  # Sort by end rank



In [33]:
# Run the function with x = 2 and the third week of January, 2023
get_gaining_rank_by_cryptos_2(2, '2023-01-15')

{'edgecoin-2': (871.0, 157.0),
 'klaycity-orb': (820.0, 166.0),
 'nucypher': (206.0, 177.0),
 'synapse-2': (212.0, 192.0),
 'gains-farm': (207.0, 193.0),
 'constellation-labs': (210.0, 196.0),
 'swipe': (205.0, 199.0)}

In [34]:
import pprint as pp
# outside of top 200 to inside top 200
a = get_gaining_rank_by_cryptos(2, '2023-02-15')
pp.pprint(a)


{'access-protocol': {'2023-02-15': 889.0, '2023-02-21': 141.0},
 'alchemy-pay': {'2023-02-15': 340.0, '2023-02-21': 197.0},
 'biconomy': {'2023-02-15': 205.0, '2023-02-21': 192.0},
 'conflux-token': {'2023-02-15': 253.0, '2023-02-21': 78.0},
 'nervos-network': {'2023-02-15': 245.0, '2023-02-21': 198.0}}


In [ ]:

# outside of top 200 to inside top 100 - top 200
b = get_gaining_rank_by_cryptos_2(2, '2023-02-15') 
pp.pprint(b)

In [36]:
# Load the necessary libraries and data again
import pandas as pd
from datetime import timedelta

# Load the CSV data
data = pd.read_csv('./ranked_by_data.csv')

# Initialize empty lists to hold the date, cryptocurrency name, and market cap
dates = []
cryptos = []
market_caps = []

# Loop over each date column in the dataframe
for col in data.columns[1:]:
    # Loop over each row in the column
    for val in data[col]:
        # Split the tuple into its two components
        crypto, market_cap = eval(val)
        # Append the data to the lists
        dates.append(col)
        cryptos.append(crypto)
        market_caps.append(float(market_cap))

# Create a new dataframe with the transformed data
df = pd.DataFrame({
    'Date': pd.to_datetime(dates),
    'Crypto': cryptos,
    'MarketCap': market_caps
})

df = df.sort_values(by=['Date', 'MarketCap'], ascending=[True, False])

# Calculate the rank of each cryptocurrency on each date
df['Rank'] = df.groupby('Date')['MarketCap'].rank(method='first', ascending=False)


In [73]:
data.columns[0]

'Unnamed: 0'

In [65]:
len(cryptos)

2021400

In [66]:
len(market_caps)

2021400

In [37]:
# Redefine the function to include the highest and lowest rank over the lifetime of each cryptocurrency
def get_gaining_rank_by_cryptos_interval(x, start_date, end_date):
    # Convert start_date and end_date to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Filter the data to only include the specified interval
    interval_data = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

    # Find the cryptocurrencies that were outside the top y*100 at the start of the interval
    outside_top_y_start = interval_data[interval_data['Date'] == start_date]
    outside_top_y_start = outside_top_y_start[outside_top_y_start['Rank'] > x*100]['Crypto'].unique()

    # Find the cryptocurrencies that were inside the top x*100 at the end of the interval
    inside_top_x_end = interval_data[interval_data['Date'] == end_date]
    inside_top_x_end = inside_top_x_end[inside_top_x_end['Rank'] <= x*100]['Crypto'].unique()

    # Find the cryptocurrencies that were outside the top y*100 at the start of the interval but inside the top x*100 at the end
    entered_top_x = [crypto for crypto in outside_top_y_start if crypto in inside_top_x_end]

    # Get rank data for each crypto that entered the top x*100
    crypto_ranks = {}
    for crypto in entered_top_x:
        crypto_data = df[df['Crypto'] == crypto]
        highest_rank_date = crypto_data['Rank'].idxmin()
        lowest_rank_date = crypto_data['Rank'].idxmax()
        crypto_ranks[crypto] = {
            "rank_start": {str(start_date.date()): interval_data[(interval_data['Crypto'] == crypto) & (interval_data['Date'] == start_date)]['Rank'].values[0]},
            "rank_end": {str(end_date.date()): interval_data[(interval_data['Crypto'] == crypto) & (interval_data['Date'] == end_date)]['Rank'].values[0]},
            "rank_highest": {str(crypto_data.loc[highest_rank_date, 'Date'].date()): crypto_data.loc[highest_rank_date, 'Rank']} if pd.notnull(crypto_data.loc[highest_rank_date, 'Rank']) else "No data",
            "rank_lowest": {str(crypto_data.loc[lowest_rank_date, 'Date'].date()): crypto_data.loc[lowest_rank_date, 'Rank']}
        }

    return {k: v for k, v in sorted(crypto_ranks.items(), key=lambda item: item[1]["rank_end"][str(end_date.date())])}  # Sort by end rank

In [38]:
# Run the function with x = 2 and the interval from January 15, 2023, to February 15, 2023
get_gaining_rank_by_cryptos_interval(2, '2023-01-15', '2023-02-15')

{'whitebit': {'rank_start': {'2023-01-15': 876.0},
  'rank_end': {'2023-02-15': 81.0},
  'rank_highest': {'2022-10-21': 35.0},
  'rank_lowest': {'2022-03-31': 900.0}},
 'edgecoin-2': {'rank_start': {'2023-01-15': 877.0},
  'rank_end': {'2023-02-15': 106.0},
  'rank_highest': {'2023-02-26': 89.0},
  'rank_lowest': {'2022-04-12': 900.0}},
 'wemix-token': {'rank_start': {'2023-01-15': 250.0},
  'rank_end': {'2023-02-15': 134.0},
  'rank_highest': {'2023-02-23': 79.0},
  'rank_lowest': {'2022-03-01': 900.0}},
 'synapse-2': {'rank_start': {'2023-01-15': 212.0},
  'rank_end': {'2023-02-15': 144.0},
  'rank_highest': {'2022-05-09': 113.0},
  'rank_lowest': {'2021-05-04': 900.0}},
 'injective-protocol': {'rank_start': {'2023-01-15': 215.0},
  'rank_end': {'2023-02-15': 148.0},
  'rank_highest': {'2021-01-23': 116.0},
  'rank_lowest': {'2020-08-30': 892.0}},
 'gains-farm': {'rank_start': {'2023-01-15': 207.0},
  'rank_end': {'2023-02-15': 153.0},
  'rank_highest': {'2023-02-18': 132.0},
  'rank

In [39]:
# Redefine the function to include the highest and lowest rank during the interval and over the lifetime of each cryptocurrency
def get_gaining_rank_by_cryptos_overall(x, start_date, end_date):
    # Convert start_date and end_date to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Filter the data to only include the specified interval
    interval_data = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

    # Find the cryptocurrencies that were outside the top y*100 at the start of the interval
    outside_top_y_start = interval_data[interval_data['Date'] == start_date]
    outside_top_y_start = outside_top_y_start[outside_top_y_start['Rank'] > x*100]['Crypto'].unique()

    # Find the cryptocurrencies that were inside the top x*100 at the end of the interval
    inside_top_x_end = interval_data[interval_data['Date'] == end_date]
    inside_top_x_end = inside_top_x_end[inside_top_x_end['Rank'] <= x*100]['Crypto'].unique()

    # Find the cryptocurrencies that were outside the top y*100 at the start of the interval but inside the top x*100 at the end
    entered_top_x = [crypto for crypto in outside_top_y_start if crypto in inside_top_x_end]

    # Get rank data for each crypto that entered the top x*100
    crypto_ranks = {}
    for crypto in entered_top_x:
        crypto_data = df[df['Crypto'] == crypto]
        interval_crypto_data = interval_data[interval_data['Crypto'] == crypto]
        highest_rank_date_lifetime = crypto_data['Rank'].idxmin()
        lowest_rank_date_lifetime = crypto_data['Rank'].idxmax()
        highest_rank_date_interval = interval_crypto_data['Rank'].idxmin()
        lowest_rank_date_interval = interval_crypto_data['Rank'].idxmax()
        crypto_ranks[crypto] = {
            "rank_start": {str(start_date.date()): interval_crypto_data[interval_crypto_data['Date'] == start_date]['Rank'].values[0]},
            "rank_end": {str(end_date.date()): interval_crypto_data[interval_crypto_data['Date'] == end_date]['Rank'].values[0]},
            "rank_highest_during_interval": {str(interval_crypto_data.loc[highest_rank_date_interval, 'Date'].date()): interval_crypto_data.loc[highest_rank_date_interval, 'Rank']} if pd.notnull(interval_crypto_data.loc[highest_rank_date_interval, 'Rank']) else "No data",
            "rank_lowest_during_interval": {str(interval_crypto_data.loc[lowest_rank_date_interval, 'Date'].date()): interval_crypto_data.loc[lowest_rank_date_interval, 'Rank']},
            "rank_highest_lifetime": {str(crypto_data.loc[highest_rank_date_lifetime, 'Date'].date()): crypto_data.loc[highest_rank_date_lifetime, 'Rank']} if pd.notnull(crypto_data.loc[highest_rank_date_lifetime, 'Rank']) else "No data",
            "rank_lowest_lifetime": {str(crypto_data.loc[lowest_rank_date_lifetime, 'Date'].date()): crypto_data.loc[lowest_rank_date_lifetime, 'Rank']}
        }

    return {k: v for k, v in sorted(crypto_ranks.items(), key=lambda item: item[1]["rank_end"][str(end_date.date())])}  # Sort by end rank


{'whitebit': {'rank_start': {'2023-01-15': 876.0},
  'rank_end': {'2023-02-15': 81.0},
  'rank_highest': {'2022-10-21': 35.0},
  'rank_lowest': {'2022-03-31': 900.0}},
 'edgecoin-2': {'rank_start': {'2023-01-15': 877.0},
  'rank_end': {'2023-02-15': 106.0},
  'rank_highest': {'2023-02-26': 89.0},
  'rank_lowest': {'2022-04-12': 900.0}},
 'wemix-token': {'rank_start': {'2023-01-15': 250.0},
  'rank_end': {'2023-02-15': 134.0},
  'rank_highest': {'2023-02-23': 79.0},
  'rank_lowest': {'2022-03-01': 900.0}},
 'synapse-2': {'rank_start': {'2023-01-15': 212.0},
  'rank_end': {'2023-02-15': 144.0},
  'rank_highest': {'2022-05-09': 113.0},
  'rank_lowest': {'2021-05-04': 900.0}},
 'injective-protocol': {'rank_start': {'2023-01-15': 215.0},
  'rank_end': {'2023-02-15': 148.0},
  'rank_highest': {'2021-01-23': 116.0},
  'rank_lowest': {'2020-08-30': 892.0}},
 'gains-farm': {'rank_start': {'2023-01-15': 207.0},
  'rank_end': {'2023-02-15': 153.0},
  'rank_highest': {'2023-02-18': 132.0},
  'rank

In [42]:
# Run the function with x = 2 and the interval from January 15, 2023, to February 15, 2023
r = get_gaining_rank_by_cryptos_overall(2, '2023-01-15', '2023-02-15')
pp.pprint(r)

{'alethea-artificial-liquid-intelligence-token': {'rank_end': {'2023-02-15': 184.0},
                                                  'rank_highest_during_interval': {'2023-02-08': 145.0},
                                                  'rank_highest_lifetime': {'2023-02-08': 145.0},
                                                  'rank_lowest_during_interval': {'2023-01-23': 268.0},
                                                  'rank_lowest_lifetime': {'2021-10-29': 900.0},
                                                  'rank_start': {'2023-01-15': 260.0}},
 'blox': {'rank_end': {'2023-02-15': 185.0},
          'rank_highest_during_interval': {'2023-02-10': 183.0},
          'rank_highest_lifetime': {'2017-12-14': 75.0},
          'rank_lowest_during_interval': {'2023-01-30': 305.0},
          'rank_lowest_lifetime': {'2022-02-22': 615.0},
          'rank_start': {'2023-01-15': 249.0}},
 'blur': {'rank_end': {'2023-02-15': 167.0},
          'rank_highest_during_interval': 

In [43]:
# Load the necessary libraries and data again
import pandas as pd
from datetime import timedelta

# Load the CSV data
data = pd.read_csv('./ranked_by_data.csv')

# Initialize empty lists to hold the date, cryptocurrency name, and market cap
dates = []
cryptos = []
market_caps = []

# Loop over each date column in the dataframe
for col in data.columns[1:]:
    # Loop over each row in the column
    for val in data[col]:
        # Split the tuple into its two components
        crypto, market_cap = eval(val)
        # Append the data to the lists
        dates.append(col)
        cryptos.append(crypto)
        market_caps.append(float(market_cap))

In [74]:
import dtale
d = dtale.show(data)
d.open_browser()

In [86]:
d = dtale.show(data.iloc[:, [0, 1, 2, 104, 1133, 2210]])

In [87]:
d.open_browser()

In [92]:
# Load the necessary libraries and data again
import pandas as pd
from datetime import timedelta
import numpy as np

# Load the CSV data
data = pd.read_csv('./ranked_by_data.csv')

# Initialize empty lists to hold the date, cryptocurrency name, and market cap
dates = []
cryptos = []
market_caps = []

# Initialize dictionaries to hold the origin date and market cap for each cryptocurrency
crypto_origin = {}
crypto_origin_marketcap = {}

# Loop over each date column in the dataframe
for col in data.columns[1:]:
    # Loop over each row in the column
    for val in data[col]:
        # Split the tuple into its two components
        crypto, market_cap = eval(val)
        # Append the data to the lists
        dates.append(col)
        cryptos.append(crypto)
        market_caps.append(float(market_cap))
        # Update the origin date and market cap for the cryptocurrency if it has not been set yet or if the current date is earlier
        if crypto not in crypto_origin or pd.to_datetime(col) < crypto_origin[crypto]:
            crypto_origin[crypto] = pd.to_datetime(col)
            crypto_origin_marketcap[crypto] = float(market_cap)

# Create a new dataframe with the transformed data
df = pd.DataFrame({
    'Date': pd.to_datetime(dates),
    'Crypto': cryptos,
    'MarketCap': market_caps
})

df = df.sort_values(by=['Date', 'MarketCap'], ascending=[True, False])

print('hello')
# Filter out rows where market cap is 0
df = df[df['MarketCap'] != 0.0]
print('hey')

# Calculate the rank of each cryptocurrency on each date
df['Rank'] = df.groupby('Date')['MarketCap'].rank(method='first', ascending=False)

KeyboardInterrupt: 

Exception ignored in: 'pandas._libs.tslibs.strptime.format_is_iso'
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.11/3.11.4/Frameworks/Python.framework/Versions/3.11/lib/python3.11/re/__init__.py", line 163, in match
    def match(pattern, string, flags=0):
    
KeyboardInterrupt: 


: 

: 

In [ ]:
# Modify the get_gaining_rank_by_cryptos function
def get_gaining_rank_by_cryptos(x, start_date, end_date):
    # Convert dates to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Filter the data to only include the specified interval
    interval_data = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    lifetime_data = df[(df['Date'] <= end_date)]
    
    # Find the cryptocurrencies that were outside the top x*100 at the start of the interval
    outside_top_x_start = interval_data[interval_data['Date'] == start_date]
    outside_top_x_start = outside_top_x_start[outside_top_x_start['Rank'] > x*100]['Crypto'].unique()
    
    # Find the cryptocurrencies that were inside the top x*100 at the end of the interval
    inside_top_x_end = interval_data[interval_data['Date'] == end_date]
    inside_top_x_end = inside_top_x_end[inside_top_x_end['Rank'] <= x*100]['Crypto'].unique()
    
    # Find the cryptocurrencies that were outside the top x*100 at the start of the interval but inside the top x*100 at the end
    entered_top_x = [crypto for crypto in outside_top_x_start if crypto in inside_top_x_end]
    
    # Get previous rank and end rank for each crypto that entered the top x*100
    crypto_info = {}
    for crypto in entered_top_x:
        crypto_interval_data = interval_data[interval_data['Crypto'] == crypto]
        crypto_lifetime_data = lifetime_data[lifetime_data['Crypto'] == crypto]
        crypto_info[crypto] = {
            "rank_start": {start_date.date(): crypto_interval_data[crypto_interval_data['Date'] == start_date]['Rank'].values[0]},
            "rank_end": {end_date.date(): crypto_interval_data[crypto_interval_data['Date'] == end_date]['Rank'].values[0]},
            "rank_highest_during_interval": {crypto_interval_data.loc[crypto_interval_data['Rank'].idxmin(), 'Date'].date(): crypto_interval_data['Rank'].min()},
            "rank_lowest_during_interval": {crypto_interval_data.loc[crypto_interval_data['Rank'].idxmax(), 'Date'].date(): crypto_interval_data['Rank'].max()},
            "rank_highest_lifetime": {crypto_lifetime_data.loc[crypto_lifetime_data['Rank'].idxmin(), 'Date'].date(): crypto_lifetime_data['Rank'].min()},
            "rank_lowest_lifetime": {crypto_lifetime_data.loc[crypto_lifetime_data['Rank'].idxmax(), 'Date'].date(): crypto_lifetime_data['Rank'].max()},
            "crypto_origin": {crypto_origin[crypto].date(): crypto_origin_marketcap[crypto]}
        }
    return crypto_info